In [ ]:
import cv2
import numpy as np
import time
import mediapipe as mp
from keras_facenet import FaceNet
import os

# Init MediaPipe face mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)

# Init FaceNet
facenet = FaceNet()

# Utility: compute Euclidean distance
def distance(a, b):
    return np.linalg.norm(a - b)

# Load and embed reference image
def get_reference_embedding(path):
    if not os.path.exists(path):
        print("Reference image not found.")
        return None

    img = cv2.imread(path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(img_rgb)

    if not results.multi_face_landmarks:
        print("No face found in reference image.")
        return None

    # Bounding box
    h, w, _ = img.shape
    landmarks = results.multi_face_landmarks[0]
    xs = [lm.x for lm in landmarks.landmark]
    ys = [lm.y for lm in landmarks.landmark]
    xmin, xmax = int(min(xs) * w), int(max(xs) * w)
    ymin, ymax = int(min(ys) * h), int(max(ys) * h)

    face_crop = img_rgb[ymin:ymax, xmin:xmax]
    face_crop = cv2.resize(face_crop, (160, 160))
    face_crop = np.expand_dims(face_crop, axis=0)

    return facenet.embeddings(face_crop)[0]

# Eye aspect ratio to detect blink
def eye_aspect_ratio(landmarks, left_indices, right_indices):
    def get_point(i): return np.array([landmarks[i].x, landmarks[i].y])
    left_eye = [get_point(i) for i in left_indices]
    right_eye = [get_point(i) for i in right_indices]
    # vertical and horizontal distances
    def aspect(e): return np.linalg.norm(e[1]-e[5]) / (2.0*np.linalg.norm(e[0]-e[3]))
    return aspect(left_eye), aspect(right_eye)

# Reference image path
reference_image_path = input("Enter path to reference image: ")
reference_embedding = get_reference_embedding(reference_image_path)
if reference_embedding is None:
    exit()

# Actions to perform for liveness
actions = ["blink", "look left", "look right", "look up", "look down"]
action_index = 0
action_done = {act: False for act in actions}

print("\nMatch successful! Now perform the following actions for liveness check:\n")

cap = cv2.VideoCapture(0)
threshold = 0.7
cooldown = 1.5
last_action_time = time.time()

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        continue

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(frame_rgb)

    if result.multi_face_landmarks:
        h, w, _ = frame.shape
        landmarks = result.multi_face_landmarks[0].landmark

        # Get bounding box
        xs = [lm.x for lm in landmarks]
        ys = [lm.y for lm in landmarks]
        xmin, xmax = int(min(xs) * w), int(max(xs) * w)
        ymin, ymax = int(min(ys) * h), int(max(ys) * h)

        # Face match
        face_crop = frame_rgb[ymin:ymax, xmin:xmax]
        try:
            face_crop = cv2.resize(face_crop, (160, 160))
            face_crop = np.expand_dims(face_crop, axis=0)
            live_embedding = facenet.embeddings(face_crop)[0]
            dist = distance(reference_embedding, live_embedding)
            match = dist < threshold
        except:
            match = False

        if match:
            current_action = actions[action_index]

            # Landmark indices for eyes
            LEFT_EYE = [33, 160, 158, 133, 153, 144]
            RIGHT_EYE = [362, 385, 387, 263, 373, 380]

            # Get nose & eyes positions
            nose = landmarks[1]
            nose_x, nose_y = nose.x * w, nose.y * h
            left_ear = landmarks[234].x * w
            right_ear = landmarks[454].x * w

            # Liveness actions
            if current_action == "blink":
                left_ear_ratio, right_ear_ratio = eye_aspect_ratio(landmarks, LEFT_EYE, RIGHT_EYE)
                if left_ear_ratio < 0.25 and right_ear_ratio < 0.25:
                    if time.time() - last_action_time > cooldown:
                        action_done[current_action] = True
                        action_index += 1
                        last_action_time = time.time()

            elif current_action == "look left" and nose_x < left_ear:
                if time.time() - last_action_time > cooldown:
                    action_done[current_action] = True
                    action_index += 1
                    last_action_time = time.time()

            elif current_action == "look right" and nose_x > right_ear:
                if time.time() - last_action_time > cooldown:
                    action_done[current_action] = True
                    action_index += 1
                    last_action_time = time.time()

            elif current_action == "look up" and nose_y < h // 3:
                if time.time() - last_action_time > cooldown:
                    action_done[current_action] = True
                    action_index += 1
                    last_action_time = time.time()

            elif current_action == "look down" and nose_y > (2 * h) // 3:
                if time.time() - last_action_time > cooldown:
                    action_done[current_action] = True
                    action_index += 1
                    last_action_time = time.time()

            cv2.putText(frame, f"Action: {current_action}", (30, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        else:
            cv2.putText(frame, "Face does not match reference", (30, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Draw box
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0) if match else (0, 0, 255), 2)

        # Show status
        y_offset = 70
        for act in actions:
            color = (0, 255, 0) if action_done[act] else (0, 0, 255)
            cv2.putText(frame, f"{act}: {'✔' if action_done[act] else '✖'}",
                        (30, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
            y_offset += 30

        # All actions done
        if all(action_done.values()):
            cv2.putText(frame, "LIVENESS VERIFIED ✅", (30, y_offset + 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 255, 0), 3)

    cv2.imshow("Liveness Verification", frame)
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import numpy as np
from mtcnn import MTCNN
from keras_facenet import FaceNet
import os

# Initialize face detector and FaceNet model
detector = MTCNN()
facenet = FaceNet()

# Function to load and preprocess an image
def load_and_preprocess_image(image_path):
    if not os.path.exists(image_path):
        print(f"Error: Image '{image_path}' not found!")
        return None

    img = cv2.imread(image_path)
    
    if img is None:
        print("Error: OpenCV could not read the image. Try another format.")
        return None

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(img_rgb)
    
    if len(faces) == 0:
        print("No face detected in the reference image.")
        return None

    x, y, w, h = faces[0]['box']
    face = img_rgb[y:y+h, x:x+w]
    face = cv2.resize(face, (160, 160))  # Resize for FaceNet
    face = np.expand_dims(face, axis=0)  # Expand dims for model input
    
    return facenet.embeddings(face)[0]  # Get feature embedding

# Prompt user to enter the reference image path
reference_image_path = input("Enter the path of the reference image: ")

# Load and process the reference image
reference_embedding = load_and_preprocess_image(reference_image_path)

if reference_embedding is None:
    exit()

# Open webcam and compare faces in real time
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(frame_rgb)

    for face in faces:
        x, y, w, h = face['box']
        face_crop = frame_rgb[y:y+h, x:x+w]
        face_crop = cv2.resize(face_crop, (160, 160))
        face_crop = np.expand_dims(face_crop, axis=0)

        # Get embedding for the real-time face
        real_time_embedding = facenet.embeddings(face_crop)[0]

        # Compute Euclidean distance between embeddings
        distance = np.linalg.norm(reference_embedding - real_time_embedding)

        # Set a threshold (lower is stricter)
        threshold = 0.7
        match_text = "Match" if distance < threshold else "No Match"

        # Draw rectangle and text
        color = (0, 255, 0) if distance < threshold else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, match_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


Enter the path of the reference image:  C:\Users\joshua benedict\Desktop\WhatsApp Image 2025-04-02 at 19.19.59_0767f0c2.jpg


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━

KeyboardInterrupt: 